In [123]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

In [124]:
import pandas as pd
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer


In [125]:
client = QdrantClient(url="http://localhost:6333")
collection_name = "no_coordinates_collection"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/home/davis/miniconda3/envs/AI/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [136]:
# I USED THIS FOR DEBUGGING
# embeddings = model.encode('900 Embarcadero Del Mar')
# print(len(embeddings))
# output = client.search(
#         collection_name=f"{collection_name}",
#         query_filter=models.Filter(
#             must=[
#                 models.FieldCondition(
#                     key="categories",
#                     match=models.MatchValue(
#                         value='Restaurants',
#                     ),
#                 )
#             ]
#         ),
#         query_vector=embeddings,
#         limit=3,
#     )
# print(output)

# Define your tools!

For this we create a Langchain tool for retrieving weather information and use the `convert_to_openai_function` to create the function definition that we will feed into the model

In [127]:
class SearchInput(BaseModel):
    location: str = Field(description="Current location of user.")
    Category: str = Field(description="Type of business that the user wants to go.")
@tool("get_category", args_schema=SearchInput)
def get_category(location: str, Category: str) -> str:
    """Get a business recommendation given location and category of user's choice.\
        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea """
    print(location, Category)
    address_embedding = model.encode(location)
    # print(address_embedding)
    output = client.search(
        collection_name=f"{collection_name}",
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="categories",
                    match=models.MatchValue(
                        value=Category,
                    ),
                )
            ]
        ),
        
        query_vector=address_embedding,
        limit=3,
    )

    print('Query output:',output)
    best_point = max(output, key=lambda x: x.score)
    restaurant_name = best_point.payload['name']
    restaurant_address = best_point.payload['full_address']

    print(f"The best-rated restaurant is {restaurant_name} located at {restaurant_address}.")

    return {"Name of restaurant": restaurant_name ,"Location": restaurant_address}

tools = [get_category]
functions = [convert_to_openai_function(t) for t in tools]
functions

[{'name': 'get_category',
  'description': 'get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. "Name of restaurant": Helena Avenue Bakery, "Location": 131 Anacapa St, Ste C Santa Barbara 93101 ,"Category": Food, Restaurants, Salad, Coffee & Tea',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'Current location of user.',
     'type': 'string'},
    'Category': {'description': 'Type of business that the user wants to go.',
     'type': 'string'}},
   'required': ['location', 'Category']}}]

# Invoke the Model

Now we can prompt the model and pass the functions 🥳

In [128]:
import ollama
import json

SYSTEM_PROMPT = f"""
You are an navigation assistant with access to these functions -
{json.dumps(functions, indent=4)}
"""

print (SYSTEM_PROMPT)


You are an navigation assistant with access to these functions -
[
    {
        "name": "get_category",
        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user's choice.        e.g. \"Name of restaurant\": Helena Avenue Bakery, \"Location\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\"Category\": Food, Restaurants, Salad, Coffee & Tea",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "Current location of user.",
                    "type": "string"
                },
                "Category": {
                    "description": "Type of business that the user wants to go.",
                    "type": "string"
                }
            },
            "required": [
                "location",
                "Category"
            ]
        }
    }
]



In [129]:
import json
import re
from typing import Optional, Dict

def parse_function_call(input_str: str) -> Optional[Dict[str, any]]:
    """
    Parses a text string to find and extract a function call.
    The function call is expected to be in the format:
    <functioncall> {"name": "<function_name>", "arguments": "<arguments_json_string>"}

    Args:
        input_str (str): The text containing the function call.

    Returns:
        Optional[Dict[str, any]]: A dictionary with 'name' and 'arguments' if a function call is found,
                                  otherwise None.
    """
    # Regex pattern to extract 'name' and 'arguments'
    pattern = r'"name":\s*"([^"]+)",\s*"arguments":\s*\'(.*?)\''

    # Search with regex
    match = re.search(pattern, input_str)
    print(match)
    if match:
        try:
            name = match.group(1)
            arguments_str = match.group(2)
            arguments = json.loads(arguments_str)
            return {"name": name, "arguments": arguments}
        except json.JSONDecodeError:
            return None
    return None


In [133]:
messages = [
     {'role': 'system','content': SYSTEM_PROMPT}, 
     {'role': 'user','content': 'Im at 900 Embarcadero Del Mar, any restaurants nearby?'},
]

response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
message = (response['message']['content'])

In [134]:
message

'<functioncall> {"name": "get_category", "arguments": \'{"location": "900 Embarcadero Del Mar", "Category": "Restaurants"}\'}'

# Call the Function

Here we pull the function out of `kwargs` and call our tool with the arguments

In [135]:
messages.append({'role': 'assistant', 'content': message}) # add ai response to history

function_call = parse_function_call(message) # parse out function call name and args into json
print(function_call)
if function_call and function_call.get("name") == "get_category":
    args = function_call.get("arguments")
    restaurant = get_category.run(args)

restaurant

<re.Match object; span=(16, 121), match='"name": "get_category", "arguments": \'{"location>
{'name': 'get_category', 'arguments': {'location': '900 Embarcadero Del Mar', 'Category': 'Restaurants'}}
900 Embarcadero Del Mar Restaurants
Query output: [ScoredPoint(id='00f1dff8-0597-58c7-a355-00ed3515c954', version=13, score=0.834775, payload={'categories': ['Burgers', 'American (Traditional)', 'Sandwiches', 'Restaurants', 'Salad', 'Soup', 'Breakfast & Brunch'], 'full_address': '900 Embarcadero Del Mar Isla Vista 93117', 'hours': {'Friday': '9:0-21:0', 'Monday': '9:0-21:0', 'Saturday': '9:0-21:0', 'Sunday': '9:0-21:0', 'Thursday': '9:0-21:0', 'Tuesday': '9:0-21:0', 'Wednesday': '9:0-21:0'}, 'name': 'Silvergreens', 'review_count': 308, 'stars': 3.5}, vector=None, shard_key=None), ScoredPoint(id='a46c0b14-71ac-59ab-8825-ddd0f1a97b17', version=51, score=0.834775, payload={'categories': ['Restaurants', 'Burgers', 'Food', 'Ice Cream & Frozen Yogurt', 'American (Traditional)', 'Salad'], 'full_add

{'Name of restaurant': 'Silvergreens',
 'Location': '900 Embarcadero Del Mar Isla Vista 93117'}

In [138]:
messages.append({'role': 'user', 'content': 'Function Response: ' + str(restaurant)})
messages

[{'role': 'system',
  'content': '\nYou are an navigation assistant with access to these functions -\n[\n    {\n        "name": "get_category",\n        "description": "get_category(location: str, Category: str) -> str - Get a business recommendation given location and category of user\'s choice.        e.g. \\"Name of restaurant\\": Helena Avenue Bakery, \\"Location\\": 131 Anacapa St, Ste C Santa Barbara 93101 ,\\"Category\\": Food, Restaurants, Salad, Coffee & Tea",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "location": {\n                    "description": "Current location of user.",\n                    "type": "string"\n                },\n                "Category": {\n                    "description": "Type of business that the user wants to go.",\n                    "type": "string"\n                }\n            },\n            "required": [\n                "location",\n                "Category"\n            ]\n

In [140]:
response = ollama.chat(model='calebfahlgren/natural-functions', messages=messages)
response['message']['content']

"The nearest restaurant is Silvergreens. It's located at 900 Embarcadero Del Mar, Isla Vista"